# Imports

In [ ]:
import multiprocessing
num_available_cpus = multiprocessing.cpu_count()

print("Number of available CPUs:", num_available_cpus)

import sys

import math
import time
import tqdm

import numpy as np
import scipy as sp
from scipy import stats

import itertools
import logging
import matplotlib.pyplot as plt

import pandas as pd
import h5py

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import torch.utils.data as utils

from argparse import ArgumentParser
import re

sys.path.append("../new_flows")
from flows import RealNVP, Planar, MAF
from models import NormalizingFlowModel

In [ ]:
from nflows.flows.base import Flow
from nflows.flows.autoregressive import MaskedAutoregressiveFlow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform, MaskedPiecewiseQuadraticAutoregressiveTransform, MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

# Load and process the data

In [ ]:
num_batches = 1

Mjj_cut = 1200
pt_cut = 550
eta_cut = None
box_cox = False

In [ ]:
num_features = 12
NS_hidden_features = 48
flow_type = 'NSQUAD' #Options are 'MAF', 'Planar' (not recommended), 'NSQUAD', and 'NSRATQUAD'

In [ ]:
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)
torch.set_default_tensor_type('torch.cuda.FloatTensor') if torch.cuda.is_available() else print ('cpu')

torch.set_num_threads(num_available_cpus)

print("Number of threads:", torch.get_num_threads())
print("Number of interop threads:", torch.get_num_interop_threads())

In [ ]:
bkg_data = np.array([])

for batch_number in range(num_batches): 
    train_batch = "/nobackup/users/myunus/CASE_samples/BB_batch%s.h5" % (batch_number)
    f = h5py.File(train_batch, "r")
    
    if batch_number == 0: 
        print("Keys: %s" % f.keys())
    
    jet_kinematics = f['jet_kinematics']
    jet1_extraInfo = f['jet1_extraInfo']
    jet2_extraInfo = f['jet2_extraInfo']
    truth_label = f['truth_label']

    np.seterr(invalid = 'ignore')

    delta_eta = jet_kinematics[:,1]

    Mjj = np.reshape(jet_kinematics[:,0], (-1,1))
    Mj1 = np.reshape(jet_kinematics[:,5], (-1,1))
    Mj2 = np.reshape(jet_kinematics[:,9], (-1,1))

    jet1_pt = np.reshape(jet_kinematics[:,2], (-1,1))
    jet2_pt = np.reshape(jet_kinematics[:,6], (-1,1))

    jet1_tau1 = np.reshape(jet1_extraInfo[:,0], (-1,1))
    jet1_tau2 = np.reshape(jet1_extraInfo[:,1], (-1,1))
    jet1_tau3 = np.reshape(jet1_extraInfo[:,2], (-1,1))
    jet1_tau4 = np.reshape(jet1_extraInfo[:,3], (-1,1))
    #jet1_btagscore = np.reshape(jet1_extraInfo[:,5],(-1,1))
    jet1_numpfconst = np.reshape(jet1_extraInfo[:,6],(-1,1))

    jet1_tau21 = jet1_tau2 / jet1_tau1
    jet1_tau32 = jet1_tau3 / jet1_tau2
    jet1_tau43 = jet1_tau4 / jet1_tau3
    jet1_sqrt_tau21 = np.sqrt(jet1_tau21) / jet1_tau1

    jet2_tau1 = np.reshape(jet2_extraInfo[:,0], (-1,1))
    jet2_tau2 = np.reshape(jet2_extraInfo[:,1], (-1,1))
    jet2_tau3 = np.reshape(jet2_extraInfo[:,2], (-1,1))
    jet2_tau4 = np.reshape(jet2_extraInfo[:,3], (-1,1))
    #jet2_btagscore = np.reshape(jet2_extraInfo[:,5],(-1,1))
    jet2_numpfconst = np.reshape(jet2_extraInfo[:,6],(-1,1))

    jet2_tau21 = jet2_tau2 / jet2_tau1
    jet2_tau32 = jet2_tau3 / jet2_tau2
    jet2_tau43 = jet2_tau4 / jet2_tau3
    jet2_sqrt_tau21 = np.sqrt(jet2_tau21) / jet2_tau1

    truth_label = truth_label[:]
    
    data = np.concatenate((Mj1, jet1_tau21, jet1_tau32, jet1_tau43, jet1_sqrt_tau21, jet1_numpfconst, 
                       Mj2, jet2_tau21, jet2_tau32, jet2_tau43, jet2_sqrt_tau21, jet2_numpfconst), axis=1)

    bkg_indices = np.where((truth_label == 0) 
                              & (Mjj > Mjj_cut) 
                              & (Mj1 > 0)
                              & (Mj2 > 0)
                              & (jet1_pt > pt_cut) 
                              & (jet2_pt > pt_cut)
                              & (np.isfinite(jet1_tau21))
                              & (np.isfinite(jet1_tau32))
                              & (np.isfinite(jet1_tau43))
                              & (np.isfinite(jet1_sqrt_tau21))
                              & (np.isfinite(jet2_tau21))
                              & (np.isfinite(jet2_tau32))
                              & (np.isfinite(jet2_tau43))
                              & (np.isfinite(jet2_sqrt_tau21)))[0]

    if eta_cut is not None: 
        bkg_eta_indices = np.where((np.abs(delta_eta) < eta_cut))[0]
        bkg_indices = np.intersect1d(bkg_indices, bkg_eta_indices)

    if batch_number == 0: 
        bkg_data = data[bkg_indices]
    else: 
        bkg_data = np.concatenate((bkg_data, data[bkg_indices]), axis=0)
    
    if box_cox: 
        
        transformed_data = np.zeros(bkg_data.shape)
        best_lambdas = []
        for col in range(num_features): 
            boxcox_col, best_lambda = stats.boxcox(bkg_data[:,col] + np.abs(np.min(bkg_data[:,col])) + 1)
            transformed_data[:,col] = boxcox_col
            best_lambdas.append(best_lambda)

        print(best_lambdas)
    
        bkg_data = transformed_data

In [ ]:
print(bkg_data.shape)

plot_var = 0
plt.hist(bkg_data[:,plot_var], bins=100)
plt.show()

In [ ]:
bkg_mean = []
bkg_std = []

for index in range(bkg_data.shape[1]):
    mean = np.mean(bkg_data[:,index])
    std = np.std(bkg_data[:,index])
    bkg_mean.append(mean)
    bkg_std.append(std)
    bkg_data[:,index] = (bkg_data[:,index]-mean)/std

In [ ]:
bkg_mean

In [ ]:
bkg_std

In [ ]:
bs = 5000 * num_batches
print("Batch Size: %s" % (bs))
    
total_PureBkg_selection = torch.tensor(bkg_data)
bkgAE_train_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs, shuffle=True) 
bkgAE_test_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs)

# Build the model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)

In [ ]:
#### MAF / Planar / NSQUAD / NSRATQUAD
class VAE_NF(nn.Module):
    def __init__(self, K, D):
        super().__init__()
        self.dim = D
        self.K = K
        
        '''
        self.encoder = nn.Sequential(
            nn.Linear(num_features, 50),
            nn.LeakyReLU(True),
            nn.Linear(50, 30),
            nn.LeakyReLU(True),
            nn.Linear(30, 20),
            nn.LeakyReLU(True),
            nn.Linear(20, D * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(D, 20),
            nn.LeakyReLU(True),
            nn.Linear(20, 30),
            nn.LeakyReLU(True),
            nn.Linear(30, 50),
            nn.LeakyReLU(True),
            nn.Linear(50, num_features)
        )
        '''
        
        self.encoder = nn.Sequential(
            nn.Linear(num_features, 30),
            nn.LeakyReLU(True),
            nn.Linear(30, 20),
            nn.LeakyReLU(True),
            nn.Linear(20, D * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(D, 20),
            nn.LeakyReLU(True),
            nn.Linear(20, 30),
            nn.LeakyReLU(True),
            nn.Linear(30, num_features)
        )
        
        if flow_type == 'NSQUAD' or flow_type == 'NSRATQUAD': 
            #----- BEGIN NEW NEURAL SPLINE CODE
            
            bkg_transforms = []
            for _ in range(K):
                bkg_transforms.append(ReversePermutation(features=D))
                if flow_type == 'NSQUAD': 
                    bkg_transforms.append(MaskedPiecewiseQuadraticAutoregressiveTransform(features=D, 
                                                                      hidden_features=NS_hidden_features, tail_bound = 3.0, tails='linear'))
                elif flow_type == 'NSRATQUAD': 
                    bkg_transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(features=D, 
                                                                      hidden_features=NS_hidden_features, tail_bound = 3.0, tails='linear'))

            #bkg_transform = CompositeTransform(bkg_transforms)
            bkg_base_dist = MultivariateNormal(torch.zeros(D).cuda(), torch.eye(D).cuda())
            self.flows = NormalizingFlowModel(bkg_base_dist, bkg_transforms)
            print(self.flows)
            
            #----- END NEW NEURAL SPLINE CODE
        
        elif flow_type == 'MAF' or flow_type == 'Planar': 
            if flow_type == 'MAF': 
                flow_init = MAF(dim=D)
            elif flow_type == 'Planar': 
                flow_init = Planar(dim=D)
            flows_init = [flow_init for _ in range(K)]
            prior = MultivariateNormal(torch.zeros(D).cuda(), torch.eye(D).cuda())
            self.flows = NormalizingFlowModel(prior, flows_init)
            print(self.flows)
        
        else: 
            print('ERROR: Flow Type not properly specified.')

    def forward(self, x):
        # Run Encoder and get NF params
        enc = self.encoder(x)
        mu = enc[:, :self.dim]
        log_var = enc[:, self.dim: self.dim * 2]

        # Re-parametrize
        sigma = (log_var * .5).exp()
        z = mu + sigma * torch.randn_like(sigma)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        # Construct more expressive posterior with NF
        
        z_k, _, sum_ladj = self.flows(z)
        
        kl_div = kl_div / x.size(0) - sum_ladj.mean()  # mean over batch

        # Run Decoder
        x_prime = self.decoder(z_k)
        return x_prime, kl_div

# Creating instance

In [ ]:
def train():
    global n_steps
    train_loss = []
    model.train()

    for batch_idx, x in enumerate(bkgAE_train_iterator):
        start_time = time.time()
        
        x = x.float().cuda()

        x_tilde, kl_div = model(x)
        
        mseloss = nn.MSELoss(size_average=False)
        
        huberloss = nn.SmoothL1Loss(size_average=False)
        
        #loss_recons = F.binary_cross_entropy(x_tilde, x, size_average=False) / x.size(0)
        loss_recons = mseloss(x_tilde,x ) / x.size(0)
        
        #loss_recons = huberloss(x_tilde,x ) / x.size(0)
        loss = loss_recons + beta * kl_div

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append([loss_recons.item(), kl_div.item()])

        if (batch_idx + 1) % PRINT_INTERVAL == 0:
            print('\tIter [{}]\tLoss: {} Time: {:5.3f} ms/batch\tbeta:{}'.format(
                batch_idx * len(x),
                np.asarray(train_loss)[-PRINT_INTERVAL:].mean(0),
                1000 * (time.time() - start_time),
                beta
            
            ))

        n_steps += 1

In [ ]:
def evaluate(split='valid'):
    global n_steps
    start_time = time.time()
    val_loss = []
    model.eval()

    with torch.no_grad():
        for batch_idx, x in enumerate(bkgAE_test_iterator):
            
            x = x.float().cuda()

            x_tilde, kl_div = model(x)
            mseloss = nn.MSELoss(size_average=False)
            #loss_recons = F.binary_cross_entropy(x_tilde, x, size_average=False) / x.size(0)
            huberloss = nn.SmoothL1Loss(size_average=False)
        

            #loss_recons = F.binary_cross_entropy(x_tilde, x, size_average=False) / x.size(0)
            loss_recons = mseloss(x_tilde,x ) / x.size(0)
            #loss_recons = huberloss(x_tilde,x ) / x.size(0)
            loss = loss_recons + beta * kl_div

            val_loss.append(loss.item())
            #writer.add_scalar('loss/{}/ELBO'.format(split), loss.item(), n_steps)
            #writer.add_scalar('loss/{}/reconstruction'.format(split), loss_recons.item(), n_steps)
            #writer.add_scalar('loss/{}/KL'.format(split), kl_div.item(), n_steps)

    print('\nEvaluation Completed ({})!\tLoss: {:5.4f}\tTime: {:5.3f} s'.format(
        split,
        np.asarray(val_loss).mean(0),
        time.time() - start_time
    ))
    
    return np.asarray(val_loss).mean(0)

In [ ]:
zdim = [4]
nflow = [10]
lrs = [1e-3]
betas = [0.0,0.01,0.05]
#betas = [0.1,0.5,1.0,2.0,10.0]
#betas = [20.0,25.0,50.0,75.0,100.0]

In [ ]:
N_EPOCHS = 1000
PRINT_INTERVAL = 50
NUM_WORKERS = 4
n_steps = 0

In [ ]:
cur_losses = []

for Z_DIM in zdim:
    for N_FLOWS in nflow:
        for beta in betas:
            model = VAE_NF(N_FLOWS, Z_DIM).cuda()
            #NOTE: The "architecture" key below can be set to "MAF", "Planar" (not recommended), "NSQUAD", or "NSRATQUAD". 
            ae_def = {
                        "type":"qcdbkg",
                        "trainon":f"etacut{re.sub('[.,]', 'p', str(eta_cut))}",
                        "features":"12features",
                        "architecture":"%s" % (flow_type),
                        "selection":"mjjcut",
                        "trainloss":"MSELoss",
                        "beta":f"beta{re.sub('[.,]', 'p', str(beta))}",
                        "zdimnflow":f"z{Z_DIM}f{N_FLOWS}"
                     }
            BEST_LOSS = 999999
            for LR in lrs:
                optimizer = optim.Adam(model.parameters(), lr=LR)
                LAST_SAVED = -1
                for epoch in range(1, N_EPOCHS + 1):
                    print("Epoch {}:".format(epoch))
                    train()
                    cur_loss = evaluate()
                    if beta == 1.0: 
                        cur_losses.append(cur_loss)

                    if cur_loss <= BEST_LOSS:
                        PATIENCE_COUNT = 0
                        BEST_LOSS = cur_loss
                        LAST_SAVED = epoch
                        print("Saving model!")
                        torch.save(model.state_dict(),f"/home/myunus/CASE/weights/{ae_def['type']}_{ae_def['trainon']}_{ae_def['features']}_{ae_def['architecture']}_{ae_def['selection']}_{ae_def['trainloss']}_{ae_def['beta']}_{ae_def['zdimnflow']}.h5")
                        #NOTE: Replace the /home/myunus/ above with the directory in which QUASAR resides. 

                    else:
                        PATIENCE_COUNT += 1
                        print("Not saving model! Last saved: {}".format(LAST_SAVED))
                        if PATIENCE_COUNT > 10:
                            print(f"############Patience Limit Reached with LR={LR}, Best Loss={BEST_LOSS}")
                            break 
                            
                model.load_state_dict(torch.load(f"/home/myunus/CASE/weights/{ae_def['type']}_{ae_def['trainon']}_{ae_def['features']}_{ae_def['architecture']}_{ae_def['selection']}_{ae_def['trainloss']}_{ae_def['beta']}_{ae_def['zdimnflow']}.h5"))
                #NOTE: Replace the /home/myunus/ above with the directory in which QUASAR resides. 

In [ ]:
plt.plot(cur_losses)
plt.xlabel('Epoch')
plt.ylabel('Bkg Loss (Beta = 1.0)')
plt.show()